In [ ]:
# ==========================================
# FASE 1: ENTRENAMIENTO DEL MODELO DE IA
# ==========================================

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. CONFIGURACIÓN
VOCAB_SIZE = 10000  # Solo usaremos las 10,000 palabras más comunes
MAX_LEN = 100       # Cortamos o rellenamos reseñas a 100 palabras

print(" Descargando y cargando el dataset IMDB...")
# Carga del dataset (ya viene dividido en entrenamiento y prueba)
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=VOCAB_SIZE)

# 2. PREPROCESAMIENTO (Padding)
# Si una reseña tiene 50 palabras, le agregamos 50 ceros. Si tiene 200, la cortamos.
print(" Normalizando longitud de las reseñas...")
x_train = pad_sequences(x_train, maxlen=MAX_LEN)
x_test = pad_sequences(x_test, maxlen=MAX_LEN)

# 3. ARQUITECTURA DE LA RED NEURONAL
print(" Construyendo el modelo...")
model = Sequential([
    Embedding(VOCAB_SIZE, 16, input_length=MAX_LEN), # Convierte números a vectores densos
    GlobalAveragePooling1D(),                        # Aplana la información (promedio)
    Dense(16, activation='relu'),                    # Capa oculta para aprender patrones
    Dense(1, activation='sigmoid')                   # Salida binaria (0=Malo, 1=Bueno)
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Resumen de lo que acabamos de construir
model.summary()

# 4. ENTRENAMIENTO
print(" Iniciando entrenamiento (esto puede tardar unos segundos)...")
history = model.fit(x_train, y_train,
                    epochs=5,              # 5 pasadas por todos los datos
                    batch_size=512,        # Procesar de 512 en 512
                    validation_data=(x_test, y_test),
                    verbose=1)

# 5. GUARDAR EL MODELO
nombre_archivo = "modelo_sentimientos.h5"
model.save(nombre_archivo)
print(f"\n ¡ÉXITO! El modelo se ha guardado como '{nombre_archivo}'")

In [ ]:
# ==========================================
# PASO EXTRA: DESCARGAR EL MODELO A TU PC
# ==========================================
from google.colab import files

try:
    files.download('modelo_sentimientos.h5')
    print("La descarga debería comenzar automáticamente.")
except Exception as e:
    print("Error al descargar: ", e)
    print("Por favor, usa el panel de la izquierda (carpeta) para descargar manualmente.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

La descarga debería comenzar automáticamente.


In [ ]:
# ==========================================
# FASE 2: DESPLIEGUE DE LA API (CORREGIDO)
# ==========================================

!pip install flask pyngrok flask-cors

import threading
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok, conf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# 1. PON TU TOKEN DIRECTAMENTE AQUÍ
# (Ya he puesto el que me pasaste para facilitarte la vida)
NGROK_TOKEN = "37LcM0paznH2qIyh92kYbKhMIKP_6Rg1Q6jaQbERBCFxYq81t"

# Configuramos ngrok directamente sin preguntar
conf.get_default().auth_token = NGROK_TOKEN

# 2. CONFIGURAR LA APP FLASK
app = Flask(__name__)
CORS(app)

print(" Cargando modelo entrenado...")
try:
    model = load_model('modelo_sentimientos.h5')
    print(" Modelo cargado exitosamente.")
except:
    print(" Error: No se encuentra 'modelo_sentimientos.h5'.")

@app.route('/')
def home():
    return "API Activa"

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        entrada = data.get('secuencia')
        if not entrada: return jsonify({'error': 'Falta secuencia'}), 400

        entrada_vectorizada = pad_sequences([entrada], maxlen=100)
        prediccion = model.predict(entrada_vectorizada)
        probabilidad = float(prediccion[0][0])
        sentimiento = "POSITIVA " if probabilidad > 0.5 else "NEGATIVA "

        return jsonify({'sentimiento': sentimiento, 'probabilidad': probabilidad})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# 3. INICIAR SERVIDOR
# Matamos procesos previos de ngrok por si acaso
ngrok.kill()

# Abrimos el túnel
public_url = ngrok.connect(5000).public_url
print(f"\n ¡LISTO! TU API ESTÁ EN VIVO")
print(f" COPIA ESTA URL PARA LA FASE 3:  {public_url}")

app.run(port=5000)

 Cargando modelo entrenado...
 Modelo cargado exitosamente.

🚀 ¡LISTO! TU API ESTÁ EN VIVO
🌍 COPIA ESTA URL PARA LA FASE 3:  https://clippingly-nonreligious-irma.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [25/Dec/2025 18:28:46] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Dec/2025 18:28:46] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
# ==========================================
# FASE 3: PROBAR LA API (CLIENTE)
# ==========================================
import requests
import json

#
url_api = "https://clippingly-nonreligious-irma.ngrok-free.dev"
endpoint = f"{url_api}/predict"

# Simulamos una reseña (ya convertida a números para simplificar)
# Imagina que estos números representan: "Esta película fue fantástica y muy emocionante"
reseña_positiva = [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100]

payload = {"secuencia": reseña_positiva}

print(f" Enviando datos a: {endpoint} ...")

try:
    response = requests.post(endpoint, json=payload)

    print(f"Estado: {response.status_code}")
    print("Respuesta del servidor:")
    print(json.dumps(response.json(), indent=2))

except Exception as e:
    print("Error al conectar:", e)